In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
from os.path import join
import pandas as pd
import numpy as np
import json
import yaml
from yaml.loader import SafeLoader

import unsafe.files as unfile
import unsafe.ddfs as unddf

# Configure

In [ ]:
# Specify FIPS, etc., 
fips_args = {
    'FIPS': ['34007'], 
    'STATEFIPS': ['34'],
    'STATEABBR': ['NJ'],
    'NATION': ['US']
}
FIPS = fips_args['FIPS'][0]
NATION = fips_args['NATION'][0]
STATEABBR = fips_args['STATEABBR'][0]
STATEFIPS = fips_args['STATEFIPS'][0]

In [ ]:
# Read in the config file and set up key parameters
ABS_DIR = Path().absolute().parents[1]

CONFIG_FILEP = join(ABS_DIR, 'config', 'config.yaml')
# Open the config file and load
with open(CONFIG_FILEP) as f:
    CONFIG = yaml.load(f, Loader=SafeLoader)

# Number of states of the world
N_SOW = CONFIG['sows']

# Hazard scenarios
SCENARIOS = CONFIG['scenarios']


In [ ]:
# Quick references to directories
FR = join(ABS_DIR, "data", "raw")

# And external - where our hazard data should be
FE = join(FR, "external")

# Set up interim and results directories as well
# We already use "FR" for raw, we use "FO" 
# because you can also think of results
# as output
FI = join(ABS_DIR, "data", "interim")
FO = join(ABS_DIR, "data", "results")

# "Raw" data directories for exposure, vulnerability (vuln) and
# administrative reference files
EXP_DIR_R = join(FR, "exp")
VULN_DIR_R = join(FR, "vuln")
REF_DIR_R = join(FR, "ref")
# Haz is for depth grids
HAZ_DIR_R = join(FE, "haz")
# Pol is for NFHL
POL_DIR_R = join(FR, "pol")

# Unzip directory 
UNZIP_DIR = join(FR, "unzipped")

# We want to process unzipped data and move it
# to the interim directory where we keep
# processed data
# Get the filepaths for unzipped data
# We unzipped the depth grids (haz) and 
# ddfs (vuln) into the "external"/ subdirectory
HAZ_DIR_UZ = join(UNZIP_DIR, "external", "haz")
POL_DIR_UZ = join(UNZIP_DIR, "pol")
REF_DIR_UZ = join(UNZIP_DIR, "ref")
VULN_DIR_UZ = join(UNZIP_DIR, "vuln")
DDF_DIR_UZ = join(UNZIP_DIR, "external", "vuln")

# "Interim" data directories
EXP_DIR_I = join(FI, "exp")
VULN_DIR_I = join(FI, "vuln")
REF_DIR_I = join(FI, "ref")
# Haz is for depth grids
HAZ_DIR_I = join(FI, "haz")
# Pol is for NFHL
POL_DIR_I = join(FI, "pol")

# Load and prepare data

In [ ]:
check_filepath = join(ABS_DIR, "data", "check", "check_structures_final.csv")
check_df = pd.read_csv(check_filepath)
drop_ids = check_df[check_df['is_house'] != True]['fd_id']

# Load the ensemble data, along with the optimal
# elevation results
sort_dfs = {}
ens_dfs = {}

for scen in SCENARIOS:
    ens_filep = join(FO, 'ensemble_' + scen + '.pqt')
    ens_df = pd.read_parquet(ens_filep)
    print('Load data: ' + scen)

    # We keep observations that we are most confident are homes
    ens_df = ens_df[~ens_df['fd_id'].isin(drop_ids)]

    opt_elev_filename = 'ens_opt_elev_' + scen + '.pqt'
    opt_elev_df = pd.read_parquet(join(EXP_DIR_I, FIPS, opt_elev_filename))
    print('Load opt elev')
    
    # Merge on fd_id and sow_ind to get eal_avoid, elev_cost, and opt_elev
    # into the ensemble
    ens_df_m = ens_df.merge(opt_elev_df,
                            on=['fd_id', 'sow_ind'],
                            suffixes=['','_opt'])

    # Add metrics for objectives that we don't have yet
    ens_df_m['rel_eal'] = ens_df_m['base_eal']/ens_df_m['val_s']
    ens_df_m['npv_opt'] = ens_df_m['pv_avoid'] - ens_df_m['elev_invst']

    ens_dfs[scen] = ens_df_m

    print('Store ensemble df\n')
    
    # We need to group by on fd_id and aggregate on our sorting columns
    sub_cols = ['pv_resid', 'npv_opt', 'fd_id', 'elev_invst',
                'avoid_rel_eal', 'rel_eal', 'val_s']
    sort_df = ens_df_m.groupby('fd_id')[sub_cols].mean()
    
    sort_dfs[scen] = sort_df

    print('Store df for sorting\n')

In [ ]:
# We also need to load in the links between structures and the
# social vulnerability data for sorting rules
sovi_filepath = join(VULN_DIR_I, 'social', FIPS, 'c_indicators.pqt')
sovi_df = pd.read_parquet(sovi_filepath)

# Allocate funding

In [ ]:
# Function for calculating the risk-burden inequality

def risk_val_gini(risk, val):
    risk = np.asarray(risk)
    val = np.asarray(val)
    # Sort both by ascending value index
    # and store the cumulative sum
    sorted_indices = np.argsort(val)
    sorted_risk = risk[sorted_indices].cumsum()
    population = np.arange(0, len(risk) + 1)/len(risk)
    # Normalize risk
    risk_norm = sorted_risk/sorted_risk.max()
    # Insert 0 for each
    risk_norm = np.insert(0., 1, risk_norm)
    # Calculate gini
    gini = np.abs((np.diff(population))*((risk_norm + np.roll(risk_norm, shift=1))[1:]/2) -
                  (np.diff(population)*(population + np.roll(population, shift=1))[1:]/2)).sum()/.5
    return gini


In [ ]:
# We will sort until we expend our budget. We get these values
# from the hma projects dataset for elevation projects
# We will sample by 500K until 6 million (roughly 95th%ile)
# then by 1M until 15 million (roughly 99th%ile)
budgets_typ = np.arange(1e6, 6.1e6, 5e5)
budgets_high = np.arange(7e6, 15.1e6, 1e6)

In [ ]:
# Columns we sort from top to bottom
h_sort_desc = ['npv_opt', 'avoid_rel_eal',
               'rel_eal']

# We loop through each scen_ddftype combo
# to execute the code below so that we have a potentially
# different set of elevated homes for each policy
# for each scenario
# Do not want to spend computation on the very high budgets
# for Lower and Upper scenario. Evaluating rules on those
# budgets is a kind of conclusion sensitivity check for
# the main 'Mid' results
for scen, sort_df in sort_dfs.items():
    # We also want to write out the ordering and
    # the allocations
    elev_dict = {}
    obj_dict = {}

    haz_scen = scen.split('_')[0]
    if haz_scen == 'Mid':
        budgets = np.append(budgets_typ, budgets_high)
    else:
        budgets = budgets_typ

    # Dict of sort keys to fd_id values
    sort_dict = {}
    # This is for community sorting, more explanation later
    slack_dict = {}
    
    print('Scenario: ' + scen)

    # Get the corresponding ensemble df
    ens_df = ens_dfs[scen]
    
    # Loop through ascending columns and sort, store in dict
    # We want to sort on the col, and give ties to lower
    # valued structures
    for col in h_sort_desc:
        sort_dict[col] = sort_df.sort_values([col, 'val_s'],
                                              ascending=[False, True]).index
    
    # Community based sorting
    sort_c_df = sort_df.join(sovi_df, how='inner')
    
    # Columns for community sorting
    c_sort_cols = ['sovi', 'lmi', 'ovb', 'cejst']
    
    # For disadvantaged communities, we will
    # spend over half of our budget on the highest
    # priority properties in that community
    # then spend the remainder of our funds
    # on any property not already elevated,
    # regardless of where it is located.
    # We prioritize based on net benefit.
    for col in c_sort_cols:
        # Households in the disadvantaged community
        sort_disad = sort_c_df[sort_c_df[col] == True]
        # Sort these by highest to lowest benefit, with
        # ties going to lower valued houses
        disad_pri = sort_disad.sort_values(['npv_opt', 'val_s'],
                                             ascending=[False, True]).index.to_list()
        # We want the community based sorting to be
        # based on greatest to lowest benefits, subject to our
        # community constraint
        sort_dict[col] = sort_df.sort_values(['npv_opt', 'val_s'],
                                               ascending=[False, True]).index.to_list()
        # We spend over half of our budget on these homes
        # before going into sort_dict[col]
        slack_dict[col] = disad_pri
    
    # Loop through budgets and the keys in sort_dict
    # Calculate the elev_inst cumulative sum and subset to
    # the value just under the budget
    # Then calculate all of the objective values
    # Store in a dict of
    # sort_key_budget keys to objectives values
    for budget in budgets:
        for sort_col, fd_id in sort_dict.items():
            # Key for obj dict
            obj_key = scen + '_' + sort_col + '_' + str(budget)
            # Sort our df according to the rule at hand
            sorted_df = sort_df.reindex(fd_id)
    
            # Calculate the cumulative sum of elev_inst
            sorted_df['policy_cost'] = sorted_df['elev_invst'].cumsum()
    
            # We want to make sure the majority of our budget
            # goes to homes in the prioritization area.
            # This appears to be the way FEMA is tracking
            # this, so we want to be consistent. 
            if sort_col in c_sort_cols:
                # The way we can do this is by taking the homes in the
                # prioritization area, and making sure that we
                # spend just over half our budget on them.
                # Then we take all the remaining homes, regardless
                # of their prioritization status, in terms of
                # top to bottom benefits. This is what is stored
                # in sorted_df. So, we just need to make sure
                # that we remove the houses elevated from
                # slack_ids before going to the remaining
                # properties.            
                
                # First we need to identify the homes that are in
                # our prioritizaton area. This is the set of homes
                # in our "slack" 
                slack_ids = slack_dict[sort_col]
                pri_df = sorted_df[sorted_df['fd_id'].isin(slack_ids)]
                
                # Next we need to allocate half of our budget
                # plus 500K (higher than our max elevaton cost ensures
                # majority of expenses go to these homes) to homes in the
                # prioritization area
                budget_alloc = budget/2 + 500000
                # We want running cost of homes we are prioritizing
                pri_df['policy_cost'] = pri_df['elev_invst'].cumsum()
                
                # Now subset based on our budget allocation limit
                elevated_sub = pri_df[pri_df['policy_cost'] <= budget_alloc]
                
                # Then we get the amount of budget we have left
                slack = budget - elevated_sub['policy_cost'].max()
                
                # Now, for all homes in sorted_df that are not in
                # elevated_sub, we subset based on slack
                slack_df = sorted_df[~sorted_df['fd_id'].isin(elevated_sub['fd_id'])]
                # Get the running cost
                slack_df['policy_cost'] = slack_df['elev_invst'].cumsum()
                
                slack_elev = slack_df[slack_df['policy_cost'] <= slack]
                
                # And we also have to subset based on the majority
                # of npv coming from our elevated_sub df
                slack_ben_max = elevated_sub['npv_opt'].sum() 
                slack_elev['npv_check'] = slack_elev['npv_opt'].cumsum()
                slack_elev_sub = slack_elev[(slack_elev['npv_check']
                                             < slack_ben_max)]
                slack_elev_sub = slack_elev_sub.drop(columns='npv_check')
                
                # Now concat
                elevated = pd.concat([elevated_sub, slack_elev_sub], axis=0)
                
            # If not community sorting, you just go through the sorted
            # dataframe and subset subject to your budget
            else:
                elevated = sorted_df[sorted_df['policy_cost'] <= budget]
    
            # Calculated objectives
            # We do this by finding the subset in ens_df that are elevated
            # and the subset that are not. That's where we calculate
            # our objectives for within each SOW, and then we take
            # the expected values across the SOWs
            elev_ens = ens_df[ens_df['fd_id'].isin(elevated['fd_id'])]
            orig_ens = ens_df[~ens_df['fd_id'].isin(elevated['fd_id'])]
    
            # For npv, we calculate the sum of npv of elevated homes
            npvs = elev_ens.groupby('sow_ind')['npv_opt'].sum()
            # Our objective value for this policy is the mean of that
            npv = np.mean(npvs)
            
            # For up_cost, we calculate the sum of elev_invst 
            up_costs = elev_ens.groupby('sow_ind')['elev_invst'].sum()
            # Then we take the mean
            up_cost = np.mean(up_costs)
    
            # Get the pv resid based on the whole set
            # of homes with risk (there are benefits out of scope
            # of our npv calculation which could be associated
            # with lowering pv of residual risk, so we want
            # policies that balance the npv of elevation while
            # also not leaving more residual risk than needed)
            
            # We calculate the sum of pv_resid for elev_ens
            # And we calculate the sum of pv_base for orig_ens
            # Since these are indexed on sow_ind, we can add the
            # two series. That's our resids, then we take the
            # mean for our resid objective value for this policy
            resid_elev = elev_ens.groupby('sow_ind')['pv_resid'].sum()
            resid_orig = orig_ens.groupby('sow_ind')['pv_base'].sum()
            resids = resid_elev + resid_orig
            resid = np.mean(resids)
    
            # Gini index between residual relative risk and structure
            # value. For elevated homes, we want to use their
            # reid_rel_eal. For homes that are not elevated, 
            # we want to use rel_eal.
            # We want to evaluate this for each SOW
            # and then take the average of the vaues
            
            # Add a column
            # with resid_rel_eal and rel_eal
            # for the corresponding homes based on elevation
            ens_df['remain_rel_eal'] = np.where(ens_df["fd_id"].isin(elevated['fd_id']),
                                                ens_df["resid_rel_eal"],
                                                ens_df["rel_eal"])

            # SOW specific gini coefficients
            ginis = ens_df.groupby('sow_ind').apply(lambda x: risk_val_gini(x['remain_rel_eal'],
                                                                            x['val_s']))
            # Ensemble mean
            gini = np.mean(ginis)

            # Finally, there are a variety of metrics on rel_eal
            # in the orig_ens dataframe
            ens_r_gb = orig_ens.groupby('sow_ind')
            # The avoid_eq objective can be calculated multiple ways
            # There is min the max resid risk burden across houses
            # There is min the median resid risk burden across houses
            # There is min the sum of resid risk burden across houses
            # There is min the 95hth %ile of resid risk burden ""
            # We can calculate it each way to show
            # there is some sensitivity about conclusions
            # based on how you calculate things
            avoid_eqs1 = ens_r_gb['rel_eal'].max()
            avoid_eq1 = np.mean(avoid_eqs1)

            avoid_eqs2 = ens_r_gb['rel_eal'].median()
            avoid_eq2 = np.mean(avoid_eqs2)

            avoid_eqs3 = ens_r_gb['rel_eal'].sum()
            avoid_eq3 = np.mean(avoid_eqs3)

            avoid_eqs4 = ens_r_gb['rel_eal'].quantile(.95)
            avoid_eq4 = np.mean(avoid_eqs4)
    
            # Store objectives in dict
            obj_dict[obj_key] = (npv, resid, up_cost,
                                 avoid_eq1,
                                 avoid_eq2,
                                 avoid_eq3,
                                 avoid_eq4,
                                 gini)
    
            # Need to store the fd_id that end up in elevated in a dict
            elev_dict[obj_key] = elevated['fd_id'].astype(int).to_list()
    
            print('Calculate objective values for policy:\n'+
                  'Sort by ' + sort_col + '\nWith Budget of $M ' + str(budget))
            
    # Get the dataframe of objectives
    # Need to have scen_policy then split into scen & policy columns
    objs = pd.DataFrame.from_dict(obj_dict).T.reset_index()
    objs.columns =  ['scen_policy', 'npv', 'pv_resid', 'up_cost',
                    'avoid_eq1', 'avoid_eq2', 'avoid_eq3', 'avoid_eq4',
                    'resid_eq']
    objs['policy'] = objs['scen_policy'].str.split('_').str[1:].apply(lambda x: '_'.join(x))
    objs['scen'] = objs['scen_policy'].str.split('_').str[:1].apply(lambda x: '_'.join(x))
    objs['sort'] = objs['policy'].str.split('_').str[:-1].apply(lambda x: '_'.join(x))
    objs['budget'] = objs['policy'].str.split('_').str[-1].astype(float).astype(int)

    # Add a community vs. household indicator
    objs.loc[objs['sort'].isin(c_sort_cols), 'res'] = 'community'
    objs.loc[~objs['sort'].isin(c_sort_cols), 'res'] = 'household'

    # Write out the dataframe of objective values
    # and the dictionary of policy to fd_ids that are
    # elevated
    obj_filep = join(FO, 'pol_obj_vals_' + scen + '.pqt')
    objs.to_parquet(obj_filep)

    elev_ids_filep = join(FO, 'pol_elev_ids_' + scen + '.json')
    with open(elev_ids_filep, 'w') as fp:
        json.dump(elev_dict, fp)